# NHIS 2010-2020 Analysis

#### Megan Hoang | HUT Script | 2-13-2022

> Data extract from IPUMS NHIS. Codebook found at: https://live.nhis.datadownload.ipums.org/web/extracts/nhis/1750331/nhis_00003.cbk

***

In [ ]:
# import all necessary modules
import pandas as pd 
import numpy as np 
import sqlite3 # for SQL queries
import csv 
#import requests # for API call
import matplotlib 
from matplotlib import pyplot as plt # import matplotlib.pyplot as plt
from matplotlib import cm #Colormap
import seaborn as sns # visualization
#import glob
import os # directory
#from sodapy import Socrata # to read in the CDC Dataset
from itertools import combinations
import statsmodels.api as sm
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.api import add_constant
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import itertools

### First, let's read in our data.

Steps in this section: 
* Set our Directory
* Use Pandas to read in the CSV
* View our data to make sure everything looks good
* View some basic summary statistics

In [ ]:
# set our directory
print(os.getcwd())
path = "/Users/meganhoang/Desktop/"
os.chdir(path)
print(os.getcwd())

In [ ]:
# read in the CSV
data = pd.read_csv("nhis_00003.csv", low_memory=False)

# let's view our data to make sure everything looks good so far
print(data.head())

In [ ]:
# let's look at some basic summary statistics
df = pd.DataFrame(data)
print(df.describe())

***
### Preprocessing

> I chose to use the sqlite3 module in python in order to use SQL queries to simplify the preprocessing process. (This way I can select the specific data I need each time.) Additionally, I chose to omit observations in the dataset that were designated as "unknown" values, for simplicity.

Steps in this section: 
* Create a SQL database
* Create the NHIS table to insert the data
* Read the CSV into the database
* Query the database for selected variables & compare sample statistics to above to make sure everything still looks OK.
* Create dummy variables and re-query

In [ ]:
# create a SQL database to store the data so we can query it
con = sqlite3.connect('nhis.db')
cur = con.cursor()

In [ ]:
# create our SQL table and insert the data
cur.execute("""create table NHIS 
            (year       INTEGER, serial      INTEGER, strata      INTEGER,
            psu         INTEGER, nhishid     INTEGER, hhweight    INTEGER,
            region      INTEGER, pernum      INTEGER, nhispid     INTEGER,
            hhx         INTEGER, fmx         INTEGER, px          INTEGER,
            perweight   INTEGER, sampweight  INTEGER, longweight  INTEGER,
            partweight  INTEGER, fweight     INTEGER, astatflg    INTEGER,
            cstatflg    INTEGER, age         INTEGER, sex         INTEGER,
            race        INTEGER, hispeth     INTEGER, lang        INTEGER,
            edu         INTEGER, pooryn      INTEGER, famincome   INTEGER, 
            health      INTEGER, mortstat    INTEGER, mortwt      INTEGER)""")

# read the csv into the database
file = open('nhis_00003.csv')
data = csv.reader(file)
cur.executemany('insert into NHIS values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', data)

# check to make sure a test query works
# cur.execute("select * from NHIS WHERE year = 2010")
# for row in cur.fetchall():
#  print(row)

# The commit method saves the changes. 
con.commit()

In [ ]:
# let's store the variables I want to query in a string:
select = """
        select
                year,
                region,
                age,
                case
                        when sex = 1 then 0
                        when sex = 2 then 1
                end as sex,
                case
                        when race = 100 or race = 200 then race
                        when race between 300 and 350 then 300
                        when race between 400 and 434 then 400
                end as race,
                case
                        when hispeth = 10 then 0
                        when hispeth between 20 and 70 then 1
                end as hispeth,
                case
                        when lang = 1 or lang = 3 then 0
                        when lang = 2 then 1
                end as lang,
                case
                        when edu between 100 and 116 then 100
                        when edu between 200 and 202 then 200
                        when edu between 300 and 303 then 300
                        when edu = 400 then edu
                        when edu between 500 and 501 then 500
                        when edu between 502 and 503 then 600
                end as edu,
                case
                        when pooryn = 1 then 0
                        when pooryn = 2 then 1
                end as pooryn,
                case
                        when famincome between 10 and 12 then 10
                        when famincome between 20 and 23 then 20
                        when famincome = 24 then 30
                end as famincome,
                health,
                mortstat 
        """

# edit the SQL query to clean the data and omit "unknown" values per IPUMS codebook
remove = """ and region < 08
        and age < 999
        and sex < 3
        and race < 500
        and hispeth < 70
        and edu < 600 
        and famincome < 96
        and health < 6"""

# remove = ""

In [ ]:
df_query = pd.read_sql_query(select + "from NHIS where year between 2010 and 2020" + remove, con)
df_query.describe()

# compare to the results from the summary statistics for df above (the non-queried dataframe)

In [ ]:
df_query = pd.read_sql_query(select + "from NHIS where year between 2010 and 2018" + remove, con)
df_query.describe()


In [ ]:
df_query = pd.read_sql_query(select + "from NHIS where year between 2019 and 2020" + remove, con)
df_query.describe()


__Dummy variables: (0 = F, 1 = T)__

* region: northeast, midwest, south (omitted: west)
* sex: female (omitted: male)
* race: white, black, native, asian (omitted: other/mixed)
    * Hisp: hisp = true (omitted: non-hispanic)
* lang: english, spanish (omitted: other)
* edu: college (omitted: no college)
* famincome: lowinc (< 50,000), midinc (between 50,000 & 100,000), highinc (> 100,000) (omitted: highinc)

__Variables left as is:__ 
* year
* age
* famincome
* health
* mortstat

> 19 variables in total: year, age, northeast, midwest, south, female, white, black, native, asian, hisp, spanish, college, lowinc, midinc, highinc


In [ ]:
# let's store the variables I want to query in a string:
# dummy variables: sex: region (West omitted) 1 = female
select_dummy = """
        select
                year,

                case 
                    when region = 01 then 1
                    else 0
                end as northeast,
                case 
                    when region = 02 then 1
                    else 0
                end as midwest,
                case 
                    when region = 03 then 1
                    else 0
                end as south,
                case
                    when region = 04 then 1
                    else 0
                end as west,

                age,

                case
                    when sex = 2 then 1
                    else 0
                end as female,


                case
                    when race = 100 then 1
                    else 0
                end as white,
                case 
                    when race = 200 then 1
                    else 0
                end as black,
                case 
                    when race = 300 then 1
                    else 0
                end as native,
                case 
                    when race = 400 then 1
                    else 0
                end as asian,
                case
                        when hispeth = 10 then 0
                        when hispeth between 20 and 70 then 1
                end as hisp,

                case
                        when lang = 1 or lang = 3 then 1
                        else 0
                end as english,
                case 
                    when lang = 2 then 1
                    else 0
                end as spanish,

                case 
                    when edu <= 202 then 1
                    else 0
                end as nocollege,
                case 
                    when edu between 300 and 399 then 1
                    else 0
                end as somecollege,
                case 
                    when edu < 400 then 0
                    when edu >= 400 then 1
                end as collegedegree,


                case
                    when famincome between 10 and 12 then 1
                    else 0
                end as lowinc,
                case
                    when famincome between 20 and 23 then 1
                    else 0
                end as midinc,
                case 
                    when famincome = 24 then 1
                    else 0
                end as highinc,
                
                health,
                mortstat 
        """

In [ ]:
df_dummy = pd.read_sql_query(select_dummy + "from NHIS where year between 2010 and 2020" + remove, con)
df_dummy.describe()

In [ ]:
# The commit method saves the changes. 
con.commit()

# Close the connection when finished. 
con.close()


***
### Visualizations

> *independent variable: health* 

Visualizations in this section: 
* Average Health per Year by Race:
    * Whites
    * Blacks
    * American Indian/Alaskan Native
    * Asian
    * Hispanic/Latino
* Average Health per Year by Gender:
    * Women
    * Men



In [ ]:
white = df_dummy.loc[df_dummy['white'] == 1, ['year', 'health']]
white.describe()

plt.figure(figsize = (10, 90))
white.groupby('year').mean().plot()
plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Whites')

#### Health by Race

In [ ]:
white = df_dummy.loc[df_dummy['white'] == 1, ['year', 'health']]
white.describe()

year_health = white.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")
for i, bar in enumerate(ax.patches):
    if i > 8:
        hatch = next(hatches)
        bar.set_hatch(hatch)
        
sns.set_style("whitegrid")
ax.set_ylim(2, 2.5)

# matplotlib.rc('xtick', labelsize=10) 
# matplotlib.rc('ytick', labelsize=10) 
# plt.rcParams.update({'font.size': 14})

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Whites')

In [ ]:
black = df_dummy.loc[df_dummy['black'] == 1, ['year', 'health']]
black.describe()

year_health = black.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Blacks')

In [ ]:
native = df_dummy.loc[df_dummy['native'] == 1, ['year', 'health']]
native.describe()

year_health = native.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i >= 0:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.55)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for American Indian/Alaskan Natives')

In [ ]:
asian = df_dummy.loc[df_dummy['asian'] == 1, ['year', 'health']]
asian.describe()

year_health = asian.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i >= 0:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.55)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Asians')

In [ ]:
hisp = df_dummy.loc[df_dummy['hisp'] == 1, ['year', 'health']]
hisp.describe()

year_health = hisp.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Hispanic/Latino')

#### Health by Gender

In [ ]:
female = df_dummy.loc[df_dummy['female'] == 1, ['year', 'health']]
female.describe()

# plt.figure(figsize = (10, 90))
# female.groupby('year').mean().plot()
# plt.xlabel('Year')
# plt.ylabel('Health')
# plt.title('Average Health per Year for Women')

year_health = female.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Women')

In [ ]:
male = df_dummy.loc[df_dummy['female'] == 0, ['year', 'health']]
male.describe()

year_health = male.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Men')

#### Health by Education

In [ ]:
college = df_dummy.loc[df_dummy['college'] == 1, ['year', 'health']]
college.describe()

year_health = college.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for College Educated')

In [ ]:
college = df_dummy.loc[df_dummy['college'] == 0, ['year', 'health']]
college.describe()

year_health = college.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Non-College Educated')

#### Health by Income

In [ ]:
low = df_dummy.loc[df_dummy['lowinc'] == 1, ['year', 'health']]
low.describe()

year_health = low.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.7)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Low-Income Families (< $50,000/yr)')

In [ ]:
mid = df_dummy.loc[df_dummy['midinc'] == 1, ['year', 'health']]
mid.describe()

year_health = mid.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.7)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Mid-Income Families (> 50k & < 100k/yr)')

In [ ]:
high = df_dummy.loc[df_dummy['highinc'] == 1, ['year', 'health']]
high.describe()

year_health = high.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(1.5, 2.7)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for High-Income Families (> $100,000/yr)')

#### Health by Interview Language

In [ ]:
english = df_dummy.loc[df_dummy['english'] == 1, ['year', 'health']]
english.describe()

year_health = english.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for English-Speakers')

In [ ]:
spanish = df_dummy.loc[df_dummy['spanish'] == 1, ['year', 'health']]
spanish.describe()

year_health = spanish.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.5)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year for Spanish-Speakers')

#### Health by Region

In [ ]:
northeast = df_dummy.loc[df_dummy['northeast'] == 1, ['year', 'health']]
northeast.describe()

year_health = northeast.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year in the Northeast')

In [ ]:
midwest = df_dummy.loc[df_dummy['midwest'] == 1, ['year', 'health']]
midwest.describe()

year_health = midwest.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year in the Midwest')

In [ ]:
south = df_dummy.loc[df_dummy['south'] == 1, ['year', 'health']]
south.describe()

year_health = south.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year in the South')

In [ ]:
west = df_dummy.loc[df_dummy['west'] == 1, ['year', 'health']]
west.describe()

year_health = west.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year in the West')

In [ ]:
#### General Health per Year

In [ ]:
all = df_dummy
all.describe()

year_health = all.groupby('year').mean().reset_index()

hatches = itertools.cycle(['//', '//'])
ax = sns.barplot(x=year_health['year'], y=year_health['health'], palette = "Blues_d")

for i, bar in enumerate(ax.patches):
    if i > 8:
        bar.set_hatch(hatch)
        hatch = next(hatches)

sns.set_style("whitegrid")
ax.set_ylim(2, 2.3)

plt.xlabel('Year')
plt.ylabel('Health')
plt.title('Average Health per Year')

***
### Models

> For my models, I chose to run two models: a Basic OLS and a Robust Linear Model (RLM) to determine some baseline coefficients. 

*independent variable: health, 
dependent variables: region, age, sex, racea, edu, pooryn, & famincome* 

Models in this section: 
* Basic OLS (from last week) -- commented out
* Basic OLS with Dummy Variables
* Robust Linear Model (from last week) -- commented out
* Robust Linear Model with Dummy Variables

In [ ]:
# now let's try fitting our RLM using the dummy variables & print summary
df_dummy = df_dummy.dropna()
x = df_dummy[['year', 'age', 'northeast', 'midwest', 'south', 'female', 'white', 'black', 'native', 'asian', 'hisp', 'spanish', 'college', 'lowinc', 'midinc']]
y = df_dummy['health']
rlm_model = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results = rlm_model.fit()
print("Parameters:")
print(rlm_results.params)
print("\n")
print(rlm_results.summary())

#### Interpretation of 'health' response variable:
> health: 1 = excellent, 2 = very good, 3 = good, 4 = fair, 5 = poor


#### Models 1.1 and 1.2 - Health v. Race

In [ ]:
# only race variables 2010-2018
df_dummy = df_dummy.dropna()
df4 = df_dummy.loc[df_dummy['year'] <= 2018]
df4.describe()

x = df4[['white', 'black', 'native', 'asian', 'hisp']]
y = df4['health']
rlm_model4 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results4 = rlm_model4.fit()
print("Parameters:")
print(rlm_results4.params)
print("\n")
print(rlm_results4.summary())

In [ ]:
# race variables 2019-2020
df_dummy = df_dummy.dropna()
df5 = df_dummy.loc[df_dummy['year'] > 2018]
df5.describe()

x = df5[['white', 'black', 'native', 'asian', 'hisp']]
y = df5['health']
rlm_model5 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results5 = rlm_model5.fit()
print("Parameters:")
print(rlm_results5.params)
print("\n")
print(rlm_results5.summary())

#### Models 2.1 and 2.2 - Socioeconomic Variables (education + income) v. Health

In [ ]:
# socioeconomic

df_dummy = df_dummy.dropna()
df6 = df_dummy.loc[df_dummy['year'] <= 2018]
df6.describe()

x = df6[['nocollege', 'somecollege' 'lowinc', 'midinc']]
y = df6['health']
rlm_model6 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results6 = rlm_model6.fit()
print("Parameters:")
print(rlm_results6.params)
print("\n")
print(rlm_results6.summary())

In [ ]:
# socioeconomic

df_dummy = df_dummy.dropna()
df7 = df_dummy.loc[df_dummy['year'] > 2018]
df7.describe()

x = df7[['college', 'lowinc', 'midinc']]
y = df7['health']
rlm_model7 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results7 = rlm_model7.fit()
print("Parameters:")
print(rlm_results7.params)
print("\n")
print(rlm_results7.summary())

#### Models 3.1 & 3.2 -- Robust Linear Model all covariates

In [ ]:
# Panel A (2010-2018)
df_dummy = df_dummy.dropna()
df2 = df_dummy.loc[df_dummy['year'] <= 2018]
df2.describe()

x = df2[['year', 'age', 'northeast', 'midwest', 'south', 'female', 'white', 'black', 'native', 'asian', 'hisp', 'spanish', 'college', 'lowinc', 'midinc']]
y = df2['health']
rlm_model2 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results2 = rlm_model2.fit()
print("Parameters:")
print(rlm_results2.params)
print("\n")
print(rlm_results2.summary())

In [ ]:
# Panel B (2019-2020)
df_dummy = df_dummy.dropna()
df3 = df_dummy.loc[df_dummy['year'] > 2018]
df3.describe()

x = df3[['year', 'age', 'northeast', 'midwest', 'south', 'female', 'white', 'black', 'native', 'asian', 'hisp', 'spanish', 'college', 'lowinc', 'midinc']]
y = df3['health']
rlm_model3 = sm.RLM(y, x, M=sm.robust.norms.HuberT())
rlm_results3 = rlm_model3.fit()
print("Parameters:")
print(rlm_results3.params)
print("\n")
print(rlm_results3.summary())